# Tarefa 2: Usar o SageMaker Experiments

Neste laboratório, você vai configurar um experimento usando o Amazon SageMaker Experiments. Você vai treinar um modelo de machine learning (ML) usando o XGBoost, executar o ajuste de hiperparâmetro para testar várias configurações de hiperparâmetros e produzir um modelo mais preciso, bem como avaliar o desempenho do modelo.

## Tarefa 2.1: Configuração do ambiente

Antes de iniciar o treinamento do modelo, instale todas as dependências necessárias.


Consulte [Gerenciar o machine learning com o Amazon SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html#experiments-features) para saber mais sobre os recursos do SageMaker Experiments.

In [ ]:
#install-dependencies

import boto3
import io
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import sagemaker
import sys
import time
import zipfile

from IPython.display import display
from IPython.display import Image
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.inputs import TrainingInput
from sagemaker.session import Session
from sagemaker.experiments.run import Run, load_run
#from sagemaker.utils import unique_name_from_base  #could be used instead of the date-time append approach, to create a unique Experiment name.
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.xgboost.estimator import XGBoost
from time import gmtime, strftime

role = sagemaker.get_execution_role()
region = boto3.Session().region_name
sess = boto3.Session()
sm = sess.client('sagemaker')
bucket = sagemaker.Session().default_bucket()
prefix = 'sagemaker/mlasms'

Depois, importe os conjuntos de dados.

In [ ]:
#import-dataset
lab_test_data = pd.read_csv('adult_data_processed.csv')
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)
lab_test_data.head()

Você vai dividir o conjunto de dados nos seguintes conjuntos de dados: treinamento (70%), validação (20%) e teste (10%). Os conjuntos de dados de treinamento e validação são usados durante o treinamento. O conjunto de dados de teste é usado na avaliação do modelo após a implantação.

Para fazer o treinamento usando o SageMaker, você precisa converter os conjuntos de dados no formato libSVM ou CSV. Este laboratório usa o formato CSV para o treinamento. 

Consulte [Algoritmo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) para saber mais sobre o algoritmo XGBoost. 
Consulte [Interface de entrada/saída do algoritmo XGBoost](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html#InputOutput-XGBoost) para saber mais sobre a interface de entrada/saída do algoritmo XGBoost.


In [ ]:
#split-dataset
train_data, validation_data, test_data = np.split(
    lab_test_data.sample(frac=1, random_state=1729),
    [int(0.7 * len(lab_test_data)), int(0.9 * len(lab_test_data))],
)

train_data.to_csv('train_data.csv', index=False, header=False)
validation_data.to_csv('validation_data.csv', index=False, header=False)

Você criou dois arquivos de conjuntos de dados, chamados *train_data.csv* e *validation_data.csv*. 
Carregue esses arquivos de conjunto de dados no Amazon Simple Storage Service (Amazon S3).

In [ ]:
#upload-dataset
from sagemaker.s3 import S3Uploader
from sagemaker.inputs import TrainingInput

sagemaker_session = sagemaker.Session()

train_path = S3Uploader.upload('train_data.csv', 's3://{}/{}'.format(bucket, prefix))
validation_path = S3Uploader.upload('validation_data.csv', 's3://{}/{}'.format(bucket, prefix))

train_input = TrainingInput(train_path, content_type='text/csv')
validation_input = TrainingInput(validation_path, content_type='text/csv')

data_inputs = {
    'train': train_input,
    'validation': validation_input
}

## Tarefa 2.2: Criar um experimento e executar um trabalho de treinamento inicial

Use o SageMaker Experiments para organizar, monitorar, comparar e avaliar os experimentos de treinamento do modelo de ML com vários componentes de treinamento. Consulte [SageMaker Experiments](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) para saber mais sobre o SageMaker Experiments. No SageMaker Experiments, esses componentes incluem conjuntos de dados, algoritmos, hiperparâmetros e métricas. 

Nesta tarefa, você vai:
- Criar e monitorar o experimento no Amazon SageMaker Studio.
- Criar uma execução para monitorar entradas, parâmetros e métricas.

Primeiro, crie um nome para o experimento e escreva uma descrição.

In [ ]:
#create unique experiment name
create_date = strftime("%m%d%H%M")

lab_6_experiment_name = "lab-6-{}".format(create_date)
description = "Using SageMaker Experiments with the Adult dataset."




Depois, defina os valores opcionais de um nome de execução e tags.

In [ ]:
# create initial run_name
run_name = "lab-6-run-{}".format(create_date)

# define a run_tag
run_tags = [{'Key': 'lab-6', 'Value': 'lab-6-run'}]

print(f"Experiment name - {lab_6_experiment_name},  run name - {run_name}")

## Tarefa 2.3: Treinar e ajustar o modelo usando o algoritmo XGBoost

O experimento está configurado e pronto para o treinamento. Quando o treinamento for concluído, você poderá analisar os resultados no SageMaker Studio. Nesta tarefa, você vai: 

- Treinar o modelo do XGBoost.
- Analisar os experimentos no SageMaker Studio.
- Ajustar o modelo com hiperparâmetros.
- Analisar os resultados do ajuste no SageMaker Studio.

### Tarefa 2.3.1: Treinar o modelo do XGBoost

Agora, treine o modelo usando o algoritmo XGBoost e o experimento que você criou. 

Os hiperparâmetros que você define são os seguintes:
- **eta**: redução do tamanho da etapa usado em atualizações para evitar sobreajuste. Após cada etapa de aumento, você pode obter diretamente os pesos dos novos recursos. 
- **gamma**: redução de perda mínima necessária para criar uma partição adicional em um nó de folha da árvore. Quanto maior, mais conservador é o algoritmo.
O parâmetro eta reduz os pesos do recurso para deixar o processo de aumento mais conservador.
- **max_depth**: profundidade máxima de uma árvore. O aumento desse valor aumenta a complexidade do modelo e a probabilidade de sobreajuste.
- **min_child_weight**: soma mínima do peso da instância (hessiano) necessária em um filho. Se a etapa de partição em árvore resultar em um nó folha com a soma do peso da instância menos min_child_weight, o processo de build não fará mais nenhum particionamento. Em modelos de regressão linear, isso corresponde a um número mínimo de instâncias necessárias em cada nó. Quanto maior o algoritmo, mais conservador ele é.
- **num_round**: o número de rodadas (árvores) usadas para o aumento. O aumento das árvores pode aumentar a precisão do modelo, mas também aumenta o risco de sobreajuste.
- **objective**: especifica a tarefa de aprendizado e o objetivo de aprendizado correspondente.
- **subsample**: taxa de subamostragem da instância de treinamento. A configuração da taxa como 0,5 significa que o XGBoost coleta aleatoriamente metade das instâncias de dados para aumentar as árvores. Isso evita o sobreajuste.
- **verbosity**: o detalhamento das mensagens de impressão. Os valores válidos são 0 (silencioso), 1 (aviso), 2 (informativo) e 3 (depuração).

O treinamento exige cerca de três a quatro minutos para ser executado.

Consulte [Hiperparâmetros](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) para saber mais sobre os hiperparâmetros do XGBoost.

In [ ]:
from sagemaker import image_uris
container = image_uris.retrieve(framework='xgboost',region=boto3.Session().region_name,version='1.5-1')

# initialize hyperparameters
eta=0.2
gamma=4
max_depth=5
min_child_weight=6
num_round=800
objective='binary:logistic'
subsample=0.8
verbosity=0

hyperparameters = {
        "max_depth":max_depth,
        "eta":eta,
        "gamma":gamma,
        "min_child_weight":min_child_weight,
        "subsample":subsample,
        "verbosity":verbosity,
        "objective":objective,
        "num_round":num_round
}

# Set up the estimator
xgb = sagemaker.estimator.Estimator(
    container,
    role, 
    instance_count=1, 
    instance_type='ml.m5.xlarge',
    output_path='s3://{}/{}/output'.format(bucket, prefix),
    sagemaker_session=sagemaker_session,
    EnableSageMakerMetricsTimeSeries=True,
    hyperparameters=hyperparameters,
    tags = run_tags
)


#Run the training job link to Experiment.
with Run(
    experiment_name=lab_6_experiment_name,
    run_name=run_name,
    tags=run_tags,
    sagemaker_session=sagemaker_session,
) as run:

    run.log_parameters({
                        "eta": eta, 
                        "gamma": gamma, 
                        "max_depth": max_depth,
                        "min_child_weight": min_child_weight,
                        "num_round": num_round,
                        "objective": objective,
                        "subsample": subsample,
                        "verbosity": verbosity
                       })
    
#    you may also specify metrics to log
#    run.log_metric(name="", value=x)

# Train the model associating the training run with the current "experiment"
    xgb.fit(
        inputs = data_inputs
    ) 

### Tarefa 2.3.2: Avaliar o desempenho do modelo antes do ajuste

No SageMaker Studio, você pode criar gráficos para avaliar os trabalhos de treinamento. Por exemplo, depois de executar os experimentos do laboratório 6, você pode examinar o valor de validation:logloss_max em um formato de gráfico.

Neste laboratório, você pode criar gráficos de métricas adicionais diretamente na linha do notebook.

In [ ]:
#visualize-training-results-table
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
plt.show()

### Tarefa 2.3.3 Ajustar o modelo com hiperparâmetros

Você realizou o treinamento do modelo usando o SageMaker Experiments com sucesso. Durante o treinamento, você também pode configurar o SageMaker para usar hiperparâmetros a fim de afetar o desempenho do modelo treinado de maneira significativa. O SageMaker Studio inclui várias opções de ajuste de hiperparâmetros comuns para o treinamento do modelo. Embora o teste de inúmeros parâmetros possa variar em eficácia dependendo do conjunto de dados usado, ele também pode exigir uma quantia significativa de tempo e esforço para criar o melhor modelo.

O ajuste automático de modelo do SageMaker automatiza a seleção de hiperparâmetros para otimizar o treinamento. Consulte [ajuste automático de modelo](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html) para saber mais sobre o ajuste automático de modelo. Para usá-lo, especifique um intervalo ou uma lista de valores possíveis para cada hiperparâmetro escolhido para o ajuste. O ajuste automático de modelo do SageMaker executa vários trabalhos de treinamento com várias configurações de hiperparâmetro. Depois, ele avalia os resultados de cada trabalho com base em uma métrica de objetivo especificada e seleciona as configurações de hiperparâmetro para novas tentativas com base nos resultados anteriores. Para cada trabalho de ajuste, você especifica um número máximo de trabalhos de treinamento e o ajuste é concluído quando esse número é atingido.

Os intervalos de hiperparâmetros que você precisa definir são os seguintes:
- **alpha**: termo de regularização L1 em pesos. O aumento desse valor deixa o modelo mais conservador.
- **eta**: redução do tamanho da etapa usado em atualizações para evitar sobreajuste. Após cada etapa de aumento, você pode obter diretamente os pesos dos novos recursos. O parâmetro eta reduz os pesos do recurso para deixar o processo de aumento mais conservador.
- **max_depth**: profundidade máxima de uma árvore. O aumento desse valor aumenta a complexidade do modelo e a probabilidade de sobreajuste.
- **min_child_weight**: soma mínima do peso da instância (hessiano) necessária em um filho. Se a etapa de partição em árvore resultar em um nó folha com a soma do peso da instância menos min_child_weight, o processo de build não fará mais nenhum particionamento. Em modelos de regressão linear, isso corresponde a um número mínimo de instâncias necessárias em cada nó. Quanto maior o algoritmo, mais conservador ele é.
- **num_round**: o número de rodadas (árvores) usadas para o aumento. O aumento das árvores pode aumentar a precisão do modelo, mas também aumenta o risco de sobreajuste.

O ajuste exige cerca de cinco minutos para ser concluído.

Consulte [intervalos de hiperparâmetros](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html) para saber mais sobre os hiperparâmetros do XGBoost.

In [ ]:
#tune-model
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

# Setup the hyperparameter ranges
hyperparameter_ranges = {
    'alpha': ContinuousParameter(0, 2),
    'eta': ContinuousParameter(0, 1),
    'max_depth': IntegerParameter(1, 10),
    'min_child_weight': ContinuousParameter(1, 10),
    'num_round': IntegerParameter(100, 1000)
}
# Define the target metric and the objective type (max/min)
objective_metric_name = 'validation:auc'
objective_type='Maximize'
# Define the HyperparameterTuner
tuner = HyperparameterTuner(
    estimator = xgb,
    objective_metric_name = objective_metric_name,
    hyperparameter_ranges = hyperparameter_ranges,
    objective_type = objective_type,
    max_jobs=12,
    max_parallel_jobs=4,
    early_stopping_type='Auto',
)

with load_run(sagemaker_session=sagemaker_session, experiment_name=lab_6_experiment_name, run_name=run_name) as run:
# Tune the model
    tuner.fit(
        inputs = data_inputs,
        job_name = lab_6_experiment_name,
    )
    run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    search_expression={
                        "Filters": [{
                                    "Name":"TrialComponentName",
                                    "Operator":"Contains",
                                    "Value":"sagemaker"
                                    }]},
    sagemaker_session=Session(sess, sm),
)
run_component_analytics.dataframe()["validation:logloss - Last"].plot(kind="bar", title="validation:logloss - Max", xlabel="training job", ylabel="logloss_max")
 
plt.show()
    

### Tarefa 2.3.4: Avaliar o desempenho do modelo após o ajuste

No SageMaker Studio, você também pode criar gráficos para avaliar os trabalhos de ajuste. Por exemplo, depois de executar o trabalho de treinamento lab-6-trial, você poderá consultar o valor de objetivo, o **validation:auc_max**, em formato de gráfico.

![Uma imagem dos gráficos validation:error_max no SageMaker Studio.](Task_2_3_4.png)

Neste laboratório, você vai exibir os resultados do melhor trabalho de ajuste e visualizá-los usando gráficos no notebook.

In [ ]:
#get_experiment_analytics 
run_component_analytics = ExperimentAnalytics(
    experiment_name=lab_6_experiment_name,
    sagemaker_session=Session(sess, sm),
)

run_component_analytics.dataframe()

In [ ]:
#visualize-tuning-results-auc-max
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    run_component_analytics.dataframe()["validation:auc - Max"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
else:
    run_component_analytics.dataframe()["validation:auc - Last"].plot(kind="bar", title="validation:auc - Max", xlabel="training job", ylabel="auc_max").set_ylim([0.8, 1]);
    
plt.show()

In [ ]:
#visualize-tuning-results-auc-max-scatter
N = 12
if run_component_analytics.dataframe()["validation:auc - Max"].iloc[1] != 0:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Max"];
else:
    x = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["validation:auc - Last"];
y = run_component_analytics.dataframe().sort_values(by=['TrialComponentName'])["num_round"]

plt.scatter(x, y, alpha=0.5)
plt.title("auc_max by num_round")
plt.xlabel("validation:auc - Max")
plt.ylabel("num_round");
plt.show()

Por fim, você poderá imprimir o melhor trabalho de ajuste com base na métrica de objetivo.

In [ ]:
#print-best
tuner.best_training_job()

### Tarefa 2.3.5: Métricas de experimento do gráfico com o SageMaker Studio usando recursos integrados

O método mencionado cria gráficos por meio de métricas de experimento usando células na linha do notebook. Uma opção adicional é criar gráficos de algumas métricas de experimento usando recursos no SageMaker Studio. Agora que o experimento já foi executado pelo menos uma vez, crie um novo gráfico de barras no SageMaker Studio.

A próxima tarefa abre uma nova aba no SageMaker Studio. Para seguir essas orientações, escolha uma das seguintes opções:
- **Opção 1:** visualizar as abas lado a lado. Para criar uma visualização de tela dividida por meio da janela principal do SageMaker Studio, arraste a aba **lab_6.ipynb** para a lateral ou escolha (clique com o botão direito) a aba **lab_6.ipynb** e selecione **New View for Notebook** (Nova visualização do notebook). Agora, as orientações ficam visíveis enquanto você explora os artefatos.
- **Opção 2:** alternar entre as abas do SageMaker Studio para seguir essas instruções. Ao terminar de explorar os artefatos, retorne ao notebook selecionando a aba **lab_6.ipynb**.

1. Selecione o ícone **Página inicial do SageMaker**.
1. Escolha **Experiments** (Experimentos).

O SageMaker Studio exibe a aba **Experiments** (Experimentos).

1. Selecione o experimento que começa com *lab-6-*.

O SageMaker Studio exibe a lista de **Runs** (Execuções) incluída nesse experimento.

1. Selecione a opção na coluna **Name** (Nome) ao lado de todas as execuções disponíveis associadas ao trabalho de ajuste de hiperparâmetro.
1. Desmarque 
1. Escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">Analisar</span>.

O SageMaker Studio exibe a aba **Run Analyze Chart** (Executar gráfico de análise).

1. Na metade inferior da aba, na seção de gráfico, escolha <span style="background-color:#1a1b22; font-size:90%; color:#57c4f8; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-width:thin; border-style:solid; border-radius:2px; margin-right:5px; white-space:nowrap">+ Add Chart (Adicionar gráfico)</span>.
1. Escolha **Bar** (Barras).

O SageMaker Studio exibe a janela **Add Chart** (Adicionar gráfico).

1. Para o **Y-axis** (Eixo Y), escolha **min_child_weight**.
1. Selecione <span style="background-color:#73cdf9; font-size:90%;  color:black; position:relative; top:-1px; padding-top:3px; padding-bottom:3px; padding-left:10px; padding-right:10px; border-color:#57c4f8; border-radius:2px; border-width:3px; margin-right:5px; white-space:nowrap">Criar</span>.

Agora, um gráfico de barras mostrando *min_child_weight* por *run* no experimento está salvo na seção de gráficos.

1. Repita esse processo e crie um novo gráfico de barras para a métrica **train:auc**.
1. Repita esse processo e crie um novo gráfico de barras para a métrica **validation:auc**.

### Conclusão

Parabéns! Você usou o SageMaker Experiments para treinar e ajustar modelos. No próximo laboratório, você vai usar o SageMaker Debugger para obter informações sobre possíveis problemas ao treinar um modelo.

## Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.